### Authorization Filters

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>نخستین مرحله از چرخه فیلترها شامل فیلترهای مجوز است. در این مرحله تصمیم گرفته می‌شود آیا کاربر اجازه‌ی دسترسی به اکشن موردنظر را دارد یا خیر. این فیلترها پیش از هر منطق دیگری اجرا می‌شوند تا در صورت عدم داشتن مجوز، اجرای اکشن را متوقف نموده و پاسخ مناسب (مانند خطای 403) را بازگردانند.</p>
</div>

every ***Authorization Filters*** must implement `IAuthorizationFilter` or `IAsyncAuthorizationFilter` 
  
`AuthorizationFilterContext` inherited from `FilterContext` and have `Result` property to return result base on my senario 

### Resource Filters

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>پس از تأیید مجوزها، نوبت به Resource Filters می‌رسد. این فیلترها <code style="font-family:vazirmatn">قبل و بعد</code> از اجرای تصمیم‌گیری درباره اکشن و فراخوانی آن اجرا می‌شوند. آن‌ها می‌توانند برای وظایفی مانند کش کردن نتایج یا ایجاد و پاکسازی منابع خاص استفاده شوند. نکته مهم این است که Resource Filters معمولاً قبل از اینکه اکشن واقعاً Resolve (توسط Model Binding) و اجرا شود، عمل می‌کنند.</p>
</div>

every ***Resource Filters*** must implement `IResourceFilter` or `IAsyncResourceFilter` 

### Action Filters

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p> این فیلترها قبل از اجرای متد اکشن و پس از اجرای متد اکشن قابل اجرا هستند. آن‌ها اجازه می‌دهند تا منطق‌هایی مثل ثبت لاگ قبل از اجرا (OnActionExecuting) و بعد از اجرا (OnActionExecuted) پیاده‌سازی شود. اگر این فیلترها تشخیص دهند که نیاز به لغو اجرای اکشن است یا باید نتیجه‌ی دیگری بازگردانند، می‌توانند این کار را انجام دهند.</p>
</div>

every ***Action Filters*** must implement `IActionFilter` or `IAsyncActionFilter` 

### Result Filters

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>هنگامی که متد اکشن اجرای خود را به پایان رساند و یک نتیجه بازگرداند، نوبت به Result Filters می‌رسد. این فیلترها قبل و بعد از اجرای نتیجه (Render شدن View یا تولید JSON یا هر پاسخ دیگر) وارد عمل می‌شوند. اگر نیاز به اعمال تغییراتی بر روی نتیجه نهایی است (مانند اضافه کردن هدرهای خاص، تغییر فرمت پاسخ، یا کش کردن خروجی)، اینجا بهترین نقطه برای این کار است.</p>
</div>

every ***Result Filters*** must implement `IResultFilter` or `IAsyncResultFilter` 

### Exception Filters

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>اگر در حین اجرای اکشن یا فیلترها استثنایی رخ دهد که هندل نشده باشد، Exception Filters وارد عمل می‌شوند. این فیلترها در زنجیره اجرا، به صورت اضطراری (در صورت وقوع خطا) فراخوانی شده و با مدیریت خطا و تولید پاسخ مناسب (مثلاً یک نتیجه‌ی خطای استاندارد یا ریدایرکت به صفحه‌ی خطا) چرخه را تکمیل می‌کنند.</p>
</div>

### Dependency Injection

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>چالش <code>Dependency Injection (DI)</code> در فیلترهای ASP.NET Core به دلیل نحوه ایجاد و استفاده از فیلترها در برنامه‌ها پیش می‌آید. برای درک این چالش و ارائه راه‌حل، بیایید ابتدا مشکل را بررسی کنیم</p>
<h3><strong>چالش Dependency Injection در فیلترها</strong></h3>
<ol><li><p><strong>مشکل در زمان‌بندی ایجاد فیلترها</strong>:</p><ul><li>فیلترها معمولاً به صورت سراسری (Global)، برای کنترلر یا اکشن خاص اعمال می‌شوند. این فیلترها ممکن است قبل از اینکه DI بتواند وابستگی‌های مورد نیاز را تزریق کند، ایجاد شوند.</li><li>اگر فیلترها به صورت مستقیم در متدهای کنترلر یا اکشن‌ها مشخص شوند (به کمک Attribute)، ASP.NET Core به جای استفاده از DI، فیلتر را به صورت مستقیم با استفاده از سازنده پیش‌فرض آن (<code>new</code>) ایجاد می‌کند. این به این معناست که وابستگی‌ها از طریق DI قابل تزریق نخواهند بود.</li></ul></li><li><p><strong>ایجاد فیلترهای Attribute-based</strong>:</p><ul><li>وقتی یک فیلتر به صورت Attribute تعریف می‌شود (مثلاً <code>[MyFilter]</code>)، ASP.NET Core از DI برای ایجاد آن استفاده نمی‌کند، بلکه آن را مستقیماً نمونه‌سازی می‌کند. این موضوع باعث می‌شود وابستگی‌های مورد نیاز فیلتر در زمان اجرا قابل تزریق نباشند.</li></ul></li><li><p><strong>وابستگی‌های پیچیده</strong>:</p><ul><li>اگر فیلتر نیاز به وابستگی‌های پیچیده یا چندگانه داشته باشد (مانند سرویس‌های ثبت‌شده در DI)، بدون دسترسی به DI، مدیریت این وابستگی‌ها دشوار می‌شود.</li></ul></li></ol>
</div>

### Global Filters

### Filters Order